In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import json
import time

from webdriver_manager.chrome import ChromeDriverManager

# Initialize the WebDriver
from selenium.webdriver.chrome.service import Service  # Add this import

# Download and set up the ChromeDriver
chrome_driver_path = ChromeDriverManager().install()
service = Service(chrome_driver_path)  # Update this line
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open the YouTube video
driver.get(video_to_scrape)

# Scroll down to load comments
for _ in range(20):  # Adjust the range to load more comments
    ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
    time.sleep(1)

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
# Click to see more replies
for button in soup.select('paper-button#more-replies'):
    driver.execute_script("arguments[0].click();", button)
    time.sleep(1)

# Stop scrolling if the number of comments matches the count-text
comment_count_text = soup.select_one('yt-formatted-string.count-text').text.strip()
total_comments = int(comment_count_text.split()[0].replace(',', ''))

if len(soup.select('#content-text')) >= total_comments:
    break
# Extract comments
comments = []
for comment in soup.select('#content-text'):
    user_id = comment.find_previous('a', {'id': 'author-text'}).span.text.strip()
    published_datetime = comment.find_previous('yt-formatted-string', {'class': 'published-time-text'}).a.text.strip()
    likes = comment.find_previous('span', {'id': 'vote-count-middle'}).text.strip()
    comment_text = comment.text.strip()
    
    comments.append({
        'user_id': user_id,
        'published_datetime': published_datetime,
        'likes': likes,
        'comment_text': comment_text
    })

# Save comments to a JSON file
with open('comments.json', 'w', encoding='utf-8') as f:
    json.dump(comments, f, ensure_ascii=False, indent=4)

# Close the WebDriver
driver.quit()
# BEGIN: Additional Scrolling and Stopping Condition
# Scroll down to load more comments until the specified number is reached
target_comment_count = 37
current_comment_count = 0

while current_comment_count < target_comment_count:
    ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    current_comment_count = len(soup.select('#content-text'))
    print(f"Current comment count: {current_comment_count}")

print("Reached the target number of comments.")
# END: Additional Scrolling and Stopping Condition

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.70)
Stacktrace:
	GetHandleVerifier [0x004A6AB3+25587]
	(No symbol) [0x00439C54]
	(No symbol) [0x00332113]
	(No symbol) [0x0030E23B]
	(No symbol) [0x003A179F]
	(No symbol) [0x003B4CB9]
	(No symbol) [0x0039A936]
	(No symbol) [0x0036BA73]
	(No symbol) [0x0036C4CD]
	GetHandleVerifier [0x00784C63+3032483]
	GetHandleVerifier [0x007D6B99+3368153]
	GetHandleVerifier [0x00538F62+624802]
	GetHandleVerifier [0x005407DC+655644]
	(No symbol) [0x0044260D]
	(No symbol) [0x0043F6D8]
	(No symbol) [0x0043F875]
	(No symbol) [0x00431CA6]
	BaseThreadInitThunk [0x76087BA9+25]
	RtlInitializeExceptionChain [0x7752C11B+107]
	RtlClearBits [0x7752C09F+191]


In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import json
import time

from webdriver_manager.chrome import ChromeDriverManager

# Initialize the WebDriver
from selenium.webdriver.chrome.service import Service  # Add this import

# Download and set up the ChromeDriver
chrome_driver_path = ChromeDriverManager().install()
service = Service(chrome_driver_path)  # Update this line
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=chrome_options)
# Open the YouTube video
video_to_scrape = "https://www.youtube.com/watch?v=Zn6-3gp8pKg" # Provide YT Video URL

driver.get(video_to_scrape)
# Scroll down to load comments
for _ in range(1):  # Adjust the range to load more comments
    ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
    time.sleep(1)
# Load the YouTube page using BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Get the variable using the provided CSS selector
comment_count_element = soup.select_one('#count yt-formatted-string span:nth-child(1)')
comment_count = comment_count_element.text if comment_count_element else 'Not found'

print(f"Comment count: {comment_count}")

Comment count: Not found


In [ ]:
%pip install webdriver_manager

In [77]:
import csv
import io
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from selenium.common import exceptions
import sys
import time

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
# Initialize the WebDriver
from selenium.webdriver.chrome.service import Service  # Add this import

def scrape(url):

    # Note: Download and replace argument with path to the driver executable.
    # Simply download the executable and move it into the webdrivers folder.
    # Download and set up the ChromeDriver
    chrome_driver_path = ChromeDriverManager().install()
    service = Service(chrome_driver_path)  # Update this line
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Navigates to the URL, maximizes the current window, and
    # then suspends execution for (at least) 5 seconds (this
    # gives time for the page to load).
    driver.get(url)
    driver.maximize_window()
    time.sleep(5)

    try:
        # Extract the elements storing the video title and
        # comment section.
        title = driver.find_element("xpath",'//*[@id="title"]/h1/yt-formatted-string').text
        comment_section = driver.find_element("xpath",'//*[@id="contents"]')
    except exceptions.NoSuchElementException:
        # Note: Youtube may have changed their HTML layouts for
        # videos, so raise an error for sanity sake in case the
        # elements provided cannot be found anymore.
        error = "Error: Double check selector OR "
        error += "element may not yet be on the screen at the time of the find operation"
        print(error)

    # Scroll into view the comment section, then allow some time
    # for everything to be loaded as necessary.
    driver.execute_script("arguments[0].scrollIntoView();", comment_section)
    time.sleep(7)

    # Scroll all the way down to the bottom in order to get all the
    # elements loaded (since Youtube dynamically loads them).
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        # Scroll down 'til "next load".
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # Wait to load everything thus far.
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # One last scroll just in case.
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    
    more_replies = driver.find_elements("xpath",'//*[@id="more-replies"]/yt-button-shape/button')
    print(len(more_replies))
    
    print(more_replies)
    for i in range(len(more_replies)):
        print('click ', i)
        driver.execute_script("arguments[0].click();", more_replies[i])
        # more_replies[i].click()
        time.sleep(1)
    
    # BEGIN: Scroll to the top of the page
    # Slowly scroll to the top of the page
    scroll_pause_time = 0.1
    current_position = driver.execute_script("return window.pageYOffset;")
    while current_position > 0:
        driver.execute_script("window.scrollTo(0, arguments[0]);", current_position - 50)
        current_position -= 50
        time.sleep(scroll_pause_time)
    # END:
    data = []
    
    try:
        time.sleep(4)
        # Extract the elements storing the usernames and comments.
        print('Extracting comments...')
        username_elems = driver.find_elements("xpath",'//*[@id="author-text"]')
        comment_elems = driver.find_elements("xpath",'//*[@id="content-text"]')
        likes_elems = driver.find_elements("xpath",'//*[@id="vote-count-middle"]')
        dates_elems = driver.find_elements("xpath",'//*[@id="published-time-text"]/a')
        
        for i in range(len(comment_elems)):
            spans = comment_elems[i].find_elements(By.TAG_NAME, "span")
            comment_text = " ".join([span.text for span in spans])
            comment_data = {
                "username": username_elems[i].text,
                "comment": comment_elems[i].text,
                "likes": likes_elems[i].text,
                "dates": dates_elems[i].text
            }
            print(i," :",comment_data)
            data.append(comment_data)
    except exceptions.NoSuchElementException:
        error = "Error: Double check selector OR "
        error += "element may not yet be on the screen at the time of the find operation"
        print(error)

    print("> VIDEO TITLE: " + title + "\n")

    # with io.open('results.csv', 'w', newline='', encoding="utf-16") as file:
    #     writer = csv.writer(file, delimiter =",", quoting=csv.QUOTE_ALL)
    #     writer.writerow(["Username", "Comment"])
    #     for username, comment in zip(username_elems, comment_elems):
    #         writer.writerow([username.text, comment.text])

    # driver.close()
   
    return data

scrapped_data = scrape("https://www.youtube.com/watch?v=Zn6-3gp8pKg")


3
[<selenium.webdriver.remote.webelement.WebElement (session="1b319a6c7164580b6d21eaafbfe23d0c", element="f.F34D633F4D765CD76C4686109436B562.d.719D9F57577030E0F562BC4FD980A60A.e.97")>, <selenium.webdriver.remote.webelement.WebElement (session="1b319a6c7164580b6d21eaafbfe23d0c", element="f.F34D633F4D765CD76C4686109436B562.d.719D9F57577030E0F562BC4FD980A60A.e.98")>, <selenium.webdriver.remote.webelement.WebElement (session="1b319a6c7164580b6d21eaafbfe23d0c", element="f.F34D633F4D765CD76C4686109436B562.d.719D9F57577030E0F562BC4FD980A60A.e.99")>]
click  0
click  1
click  2
Extracting comments...
0  : {'username': '@BlindGardener', 'comment': 'I like that you actually tried to pronounce the names correctly without American accent.', 'likes': '37', 'dates': '5 years ago'}
1  : {'username': '@Emma88178', 'comment': 'Yeah the Midwest accent can be annoying some times. But this man is not American…', 'likes': '', 'dates': '3 years ago'}
2  : {'username': '@parichehrmanuchehrkhodayaa5405', 'comm

In [72]:
len(scrapped_data)

33

In [55]:
import json

# Save scrapped_data to a JSON file
with open('scrapped_data_output.json', 'w', encoding='utf-8') as f:
    json.dump(scrapped_data, f, ensure_ascii=False, indent=4)

print("scrapped_data has been saved to scrapped_data_output.json")

scrapped_data has been saved to scrapped_data_output.json
